In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'


import os, sys, random
from pathlib import Path
import glob

from tqdm import tqdm
from fastprogress import progress_bar

import warnings
warnings.filterwarnings(action='ignore')

import pickle

CUR_DIR = Path.cwd()
DATA_DIR = CUR_DIR / 'data/preprocessed'
DATA_DIR

WindowsPath('C:/Users/안현준/풍년보험/data/preprocessed')

In [2]:
crops = ['감자', '고구마', '고추(화건)', '깻잎', '녹두(국산)', '단감', '당근(무세척)', '대파', '들깨(국산)',
       '딸기', '무', '미나리', '방울토마토', '배(신고)', '배추', '백참깨(국산)', '복숭아(백도)',
       '사과(후지)', '생강(국산)', '수박', '시금치', '애호박', '양배추', '양파', '오이(가시)', '적상추',
       '참외', '콩(흰콩국산)', '토마토', '팥(국산)', '포도(캠벨)', '풋고추']
TARGET_YEAR = [2017, 2018, 2019, 2020]

In [3]:
len(crops)

32

In [4]:
df_result = pd.DataFrame(columns=['crop', '보험가입시 평균소득', '보험미가입시 평균소득', '평균보험료', '수익률'])
volatility = pd.DataFrame(columns=['crop', 'income_without_premium_stddev', 'income_with_premium_stddev'])

for i, crop in enumerate(crops):
    cur = pd.read_csv(f'./results/{crop}/premiums_{crop}_v2.csv', encoding='utf-8-sig')
    #cur = cur[cur['year'] >= 2010]
    cur = cur[['year', f'{crop}_pred', 'MA_5_pred', '보험료_true']]
    mean_premium = cur['보험료_true'].mean() # 전체 보험료 평균
    
    # 예측가격과 5년이동평균선 중에서 최댓값을 예상소득으로 삼는다
    cur['income_pred'] = np.where(cur[f'{crop}_pred'] > cur['MA_5_pred'], cur[f'{crop}_pred'], cur['MA_5_pred'])
    
    # 2017년부터만 사용
    df_target = cur[cur.year >= TARGET_YEAR[0]].reset_index(drop=True)
    mean_income_pred = df_target.income_pred.mean()   # 평균예상소득(보험O)
    mean_crop_pred = df_target[f'{crop}_pred'].mean() # 평균예상소득(보험X)
    
    # 수익률
    pct = (mean_income_pred - mean_premium) / mean_crop_pred * 100 - 100

    df_result.loc[i] = [crop, df_target['income_pred'].mean(), df_target[f'{crop}_pred'].mean(), mean_premium, pct]
    volatility.loc[i] = [crop, df_target[f'{crop}_pred'].std(), df_target['income_pred'].std()]

In [5]:
df_result = df_result.sort_values(by=['수익률'] ,ascending=False).reset_index(drop=True)
df_result

,crop,보험가입시 평균소득,보험미가입시 평균소득,평균보험료,수익률
0,양파,21020.443508,18773.815615,178.816592,11.014337
1,들깨(국산),432758.560083,400470.872292,285.901801,7.991040
2,사과(후지),45665.998750,41364.021271,1506.060544,6.759297
3,무,16204.124692,15100.686021,221.963439,5.837319
4,애호박,23011.385100,21866.645573,273.405258,3.984764
5,생강(국산),122002.245529,109200.537437,9133.848241,3.358829
6,고추(화건),518536.700708,499891.065833,3073.986360,3.115008
7,미나리,50679.812337,49283.874771,1.486453,2.829427
8,적상추,21007.700263,20363.439396,69.286768,2.823561
9,깻잎,19607.431671,19091.612875,1.051412,2.696301


In [6]:
volatility['improved'] = (volatility['income_without_premium_stddev'] - volatility['income_with_premium_stddev']) / volatility['income_without_premium_stddev']
volatility['improved_pct'] = volatility['improved'].apply(lambda x: x*100)
volatility

,crop,income_without_premium_stddev,income_with_premium_stddev,improved,improved_pct
0,감자,2519.380922,1603.483196,0.363541,36.354079
1,고구마,3800.493154,3800.493154,0.000000,0.000000
2,고추(화건),98558.739804,65106.781798,0.339411,33.941138
3,깻잎,1191.621318,608.674533,0.489205,48.920473
4,녹두(국산),18171.140491,17134.276343,0.057061,5.706104
5,단감,3726.599409,3726.599409,0.000000,0.000000
6,당근(무세척),4652.425383,4652.425383,0.000000,0.000000
7,대파,119.200289,81.443297,0.316753,31.675252
8,들깨(국산),72818.442132,35758.305688,0.508939,50.893888
9,딸기,1301.780328,1122.205340,0.137946,13.794569


In [7]:
# 보험을 적용했을때, 변동성이 줄어든 것들의 개수
volatility[volatility['improved'] > 0].shape[0]

26